In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

In [ ]:
%cd '/content/drive/MyDrive/DIV TEKNIK INFORMATIKA/Pemrograman Sistem Cerdas - Bu Intan/Projek Sentiment Analysis'

In [ ]:
!pip install dataframe-image

In [ ]:
!pip install varname

In [ ]:
!pip install keras-metrics

In [ ]:
!pip install h5py

In [ ]:
!pip install sastrawi

In [8]:
import nltk
import pandas as pd
import re
import tensorflow as tf
import time
import multiprocessing
import io
import gensim
import numpy as np
import matplotlib.pyplot as plt
import keras_metrics as km
import pickle
import keras

from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
#from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Activation, Embedding, LSTM, Bidirectional, Dropout, GRU
from keras import regularizers
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
from nltk.tokenize import TweetTokenizer
from collections import defaultdict
from datetime import timedelta
from gensim.models import word2vec
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from varname import nameof

nltk.download('punkt')
from nltk.tokenize import word_tokenize

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
path = "review.csv"
ds = pd.read_csv(path, sep=",", header=[0], encoding="UTF-8")
ds.head()

In [ ]:
dsCont = ds['content']
dsCont

In [ ]:
dsCf = dsCont.str.lower()
dsCf.head()

In [ ]:
#Filtering
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|
\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))\n''',
 " ", content) for content in dsCf]
filtering_cont = [re.sub(r'\(cont\)'," ", content)for content in filtering_url]
filtering_punctuation = [re.sub(r'[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', content) for content in filtering_cont]
filtering_tagger = [re.sub(r'#([^\s]+)', '', content) for content in filtering_punctuation]
filtering_numeric = [re.sub(r'\d+', ' ', content) for content in filtering_tagger]
dsFilter = pd.Series(filtering_numeric)
dsFilter.head()

In [ ]:
#Tokenization
dsToken = [word_tokenize(line) for line in dsFilter]
print(dsToken)

In [14]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [15]:
factory = StopWordRemoverFactory()
ind_stopword = factory.get_stop_words()

def stopword(line):
    temp = list()
    for word in line:
       if (word not in ind_stopword):
          if(len(word)>3):
              temp.append(word)
    return temp

dsPenting = [stopword (line) for line in dsToken]
print(dsPenting)

[['error'], ['tulisan', 'page', 'unavailable', 'mengganti', 'profil', 'refresh', 'akun', 'diweb', 'bintang', 'satu'], ['usernya', 'banyak', 'toxic'], ['membuka', 'aplikasinya', 'aplikasinya', 'selalu', 'keluar', 'sendiri', 'malah', 'selalu', 'keluar', 'sedang', 'mencari', 'buku'], ['jujur', 'aplikasi', 'ribet', 'banget', 'sepertinya', 'bukan', 'aplikasi', 'membaca', 'delete', 'this', 'aplication'], ['seringkali', 'buku', 'dibaca'], ['punyaku', 'beda', 'lainn', 'padahal', 'punyaku', 'bukan', 'pick', '\u2060｡\u2060•́\u2060︿\u2060•̀\u2060｡\u2060'], ['aplikasi', 'praktis'], ['aplikasi'], ['kirain', 'kumpulan', 'buku', 'sini', 'ternyata', 'cuma', 'judul'], ['banyak', 'kasih', 'spoiler'], ['community', 'reviews'], ['nyesel', 'donlod', 'buka', 'cuman', 'judul', 'bukak', 'baca'], ['cant', 'continue', 'login', 'with', 'facebook', 'account', 'when', 'want', 'login', 'email', 'also'], ['digunain', 'cuma', 'reload', 'mulu', 'padahal', 'udah', 'update', 'versi', 'terbaru', 'signal', 'bagus'], ['hii

In [17]:
path_slang = open("kamus kata baku 1.csv")
slang = pd.read_csv(path_slang, encoding = 'utf-8', header=None, sep=";")

def replaceSlang(word):
  if word in list(slang[0]):
    indexslang = list(slang[0]).index(word)
    return slang[1][indexslang]
  else:
    return word

ulasan_formal= []
for data in dsPenting:
  ds_clean = [replaceSlang(word) for word in data]
  ulasan_formal.append(ds_clean)
#len_ulasan_formal = len(ulasan_formal)
print(ulasan_formal)
#len_ulasan_formal

[['error'], ['tulisan', 'page', 'unavailable', 'mengganti', 'profil', 'refresh', 'akun', 'diweb', 'bintang', 'satu'], ['usernya', 'banyak', 'toxic'], ['membuka', 'aplikasinya', 'aplikasinya', 'selalu', 'keluar', 'sendiri', 'malah', 'selalu', 'keluar', 'sedang', 'mencari', 'buku'], ['jujur', 'aplikasi', 'ribet', 'banget', 'sepertinya', 'bukan', 'aplikasi', 'membaca', 'delete', 'this', 'aplication'], ['seringkali', 'buku', 'dibaca'], ['punyaku', 'beda', 'lainn', 'padahal', 'punyaku', 'bukan', 'pick', '\u2060｡\u2060•́\u2060︿\u2060•̀\u2060｡\u2060'], ['aplikasi', 'praktis'], ['aplikasi'], ['diduga', 'kumpulan', 'buku', 'sini', 'ternyata', 'cuma', 'judul'], ['banyak', 'kasih', 'spoiler'], ['community', 'reviews'], ['menyesal', 'donlod', 'buka', 'cuma', 'judul', 'buka', 'baca'], ['cant', 'continue', 'login', 'with', 'facebook', 'account', 'when', 'want', 'login', 'email', 'also'], ['digunain', 'cuma', 'reload', 'terus', 'padahal', 'sudah', 'update', 'versi', 'terbaru', 'signal', 'bagus'], ['h

In [18]:
factory = StemmerFactory()
ind_stemmer = factory.create_stemmer()
def stemmer(line):
    temp = list()
    for word in line:
      #if(word not in white_list):
        word = ind_stemmer.stem(word)
      #if(len(word)>3):
        temp.append(word)
    return temp

kata_dasar = [stemmer (line) for line in ulasan_formal]
print(kata_dasar)

[['error'], ['tulis', 'page', 'unavailable', 'ganti', 'profil', 'refresh', 'akun', 'diweb', 'bintang', 'satu'], ['usernya', 'banyak', 'toxic'], ['buka', 'aplikasi', 'aplikasi', 'selalu', 'keluar', 'sendiri', 'malah', 'selalu', 'keluar', 'sedang', 'cari', 'buku'], ['jujur', 'aplikasi', 'ribet', 'banget', 'seperti', 'bukan', 'aplikasi', 'baca', 'delete', 'this', 'aplication'], ['seringkali', 'buku', 'baca'], ['punya', 'beda', 'lainn', 'padahal', 'punya', 'bukan', 'pick', ''], ['aplikasi', 'praktis'], ['aplikasi'], ['duga', 'kumpul', 'buku', 'sini', 'nyata', 'cuma', 'judul'], ['banyak', 'kasih', 'spoiler'], ['community', 'reviews'], ['sesal', 'donlod', 'buka', 'cuma', 'judul', 'buka', 'baca'], ['cant', 'continue', 'login', 'with', 'facebook', 'account', 'when', 'want', 'login', 'email', 'also'], ['digunain', 'cuma', 'reload', 'terus', 'padahal', 'sudah', 'update', 'versi', 'baru', 'signal', 'bagus'], ['hiiit', 'force', 'close'], ['jelek', 'baca'], ['masuk', 'mental', 'mental', 'mulu'], ['

In [ ]:
import json
jsons= json.dumps(kata_dasar)
jsonFile = open("data_preprocessing.json", "w")
jsonFile.write(jsons)
jsonFile.close()

fileObject = open("data_preprocessing.json", "r")
jsonContent = fileObject.read()
ds_preprocess = json.loads(jsonContent)
ds_preprocess

In [20]:
vect = TfidfVectorizer()
reviews2 = ["".join(r) for r in ds_preprocess]
vect_tfidf = vect.fit_transform(reviews2)
vect_tfidf2 = vect_tfidf.toarray()
vect_tfidf2.shape

(396, 336)

In [21]:
labels = ds['score']

In [22]:
x_train, x_test, y_train, y_test = train_test_split(reviews2, labels, test_size=0.2, random_state=42)

In [23]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

In [24]:
from sklearn.svm import LinearSVC

svm_model = LinearSVC()
svm_model.fit(vect_tfidf2, ds['score'])

LinearSVC()

In [25]:
new_text = ["Aplikasi Bagus, Keren"]
new_tfidf = vect.transform(new_text).toarray()
prediction = svm_model.predict(new_tfidf)
prediction

array([5])

In [27]:
import pickle

drive_path = '/content/drive/MyDrive/DIV TEKNIK INFORMATIKA/Pemrograman Sistem Cerdas - Bu Intan/Projek Sentiment Analysis'

# Simpan model SVM ke dalam file .pkl menggunakan pickle
model_filename = 'sentimentSVM.pkl'
with open(drive_path + model_filename, 'wb') as model_file:
    pickle.dump(svm_model, model_file)

# Simpan objek LabelEncoder ke dalam file .pkl menggunakan pickle
#label_encoder_filename = 'label_encoder.pkl'
#with open(drive_path + label_encoder_filename, 'wb') as label_encoder_file:
#    pickle.dump(label_encoder, label_encoder_file)

print(f"Model dan LabelEncoder telah disimpan sebagai {model_filename}.")

Model dan LabelEncoder telah disimpan sebagai sentimentSVM.pkl.
